In [1]:
import pandas as pd
import numpy as np

In [2]:
# Paramètres
INPUT_FICHIER_WIKI_TEXTE = "../donadas/brut/wikipedia_texte.csv"
OUTPUT_FICHER_WIKI_ETIQUETAS_AUTOMATIC = "../donadas/etiquetas/wikipedia_dialectes_automatic.csv"

MIN_LEN = 50
MAX_LEN = np.inf

DIALECTES = ["gascon", "provençal", "auvernhat", "lengadocian"]

# Ensembles de mots tipics per cada dialecte
MOTS_TIPICS = {
    "lengadocian": set(),
    "gascon": {
        "ei",
        "dab",
        "damb",
        "eth",
        "hèsta",
        "entà",
        "neishut",
        "suu",
        "peu",
        "hilh",
        "castèth",
        "medish",
        "puish",
        "har",
        "eth",
        "shens",
        "ahar",
        "enter",
    },
    "auvernhat": {
        "z-es",
        "z'es",
        "z-èra",
        "z'èra",
        "z-a",
        "z'a",
        "z-ai",
        "z'ai",
        "atí",
        "puesche",
        "ilh" "ilhs",
        "embe",
        # "dau",
    },
    "provençal": {
        "dei",
        # "lei",
        # "leis",
        "sei",
        # "dau",
        "pòu",
        "ambé",
        "elei",
        "ansin",
        "toti",
        "avans",
    },
    "francés": {
        "le",
        "les",
        "il",
        "elle",
        "je",
        "bien",
        "être",
        "avoir",
        "pour",
        "mes",
        "tes",
        "une",
        "des",
        "moi",
        "toi",
        "lui",
        "dans",
        "parce que",
        "ce",
        "cette",
        "ces",
        "même",
        "à",
        "ici",
        "aux",
        "dont",
        "était",
        "étaient",
    },
}

In [3]:
# Cargar donadas
df_tot = pd.read_csv(INPUT_FICHIER_WIKI_TEXTE, sep=",", dtype=str).set_index("Article")

In [4]:
# Filtrar tèxtes per longor
len_texte = df_tot["Texte"].apply(len)
df_sub = df_tot[(len_texte > MIN_LEN) & (len_texte < MAX_LEN)].copy()

In [5]:
def contar_mots_tipics(texte: str, mots_tipics: set) -> int:
    """
    Comptar mots tipics dins un tèxte.
    """
    return len(set(texte.lower().split()) & mots_tipics)


def custom_idxmax(row: pd.Series):
    """
    Trobar lo dialecte amb mai de mots tipics.
    """
    sorted_values = row.sort_values(ascending=False)
    if sorted_values.iloc[0] == sorted_values.iloc[1]:
        return np.nan
    else:
        return sorted_values.idxmax()


# Calcul dels mots tipics per cada dialecte
for dialecte, mots_tipics in MOTS_TIPICS.items():
    df_sub[dialecte] = df_sub["Texte"].apply(contar_mots_tipics, mots_tipics=mots_tipics)
df_sub = df_sub.loc[df_sub[DIALECTES].sum(axis=1) > 0].copy()
df_sub["Dialecte"] = df_sub[DIALECTES].apply(custom_idxmax, axis=1)

In [6]:
# Filtrar tèxtes amb mots tipics etiquetats
df_etiquetat_mots_tipics = df_sub[
    (df_sub[DIALECTES] >= 1).any(axis=1) & df_sub["Dialecte"].notna() & (df_sub["francés"] == 0)
]
df_etiquetat_mots_tipics

,Texte,lengadocian,gascon,auvernhat,provençal,francés,Dialecte
Article,,,,,,,
Vladimir_Arnold,vladimir igorevich arnold (12 de junh de 1937 ...,0,0,0,1,0,provençal
Cuelha,cuelha (queuille en francés) z-es una comuna o...,0,0,1,0,0,auvernhat
Auriòu_(Provença),"auriòu (auruou en provençau, auriol en francés...",0,0,0,1,0,provençal
Pierre-Simon_de_Laplace,"pierre-simon de laplace (23 de març de 1749, b...",0,0,0,1,0,provençal
Mirza_Ghalib,mirza asadullah khan ghalib (27 de decembre de...,0,0,0,1,0,provençal
...,...,...,...,...,...,...,...
Aurelian_(emperaire_roman),"aurelian , en latin: lucius domitius aurelianu...",0,0,0,2,0,provençal
Miranda_(Gers),miranda (mirande en francés) qu'ei ua comuna o...,0,2,0,0,0,gascon
Lekunberri_(Baisha_Navarra),"lekunberri (basco), lecumberry (francés), qu'e...",0,1,0,0,0,gascon


In [7]:
# Salvar las etiquetas automaticas
df_etiquetat_mots_tipics[["Dialecte"]].to_csv(OUTPUT_FICHER_WIKI_ETIQUETAS_AUTOMATIC)